<a href="https://colab.research.google.com/github/Pranto210102/CNN-FashionMNIST-Project-Pranto/blob/main/210102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>